In [ ]:
try:
    import chess
    import chess.engine
    import chess.pgn
    import time
    import random
    import uuid 
    import pandas as pd
    from stockfish import Stockfish
    from IPython.display import display, HTML, clear_output
    from html import escape
    from datetime import datetime
except:
    !pip install python-chess
    !pip install stockfish

In [ ]:
engine_depths = 10
iter_depths = 1
engine_skills = 1
gameFile = pd.DataFrame()

In [ ]:
stockfish = Stockfish("engine/stockfish_11.exe")
engine = chess.engine.SimpleEngine.popen_uci("engine/stockfish_11.exe")

In [ ]:
def who(player):
    return "White" if player == chess.WHITE else "Black"

In [ ]:
def display_board(board, use_svg):
    if use_svg:
        return board._repr_svg_()
    else:
        return "<pre>" + str(board) + "</pre>"

In [ ]:
def staticAnalysis(board,color):
    score = 0
    my_color = str(color)
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 5), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 12), 
                           (chess.KNIGHT, 4),
                           (chess.ROOK, 6)]:
        if my_color =='White':
            score += len(board.pieces(piece, chess.WHITE)) * value
        else:
            score += len(board.pieces(piece, not chess.WHITE)) * value
    return score

In [ ]:
class Engine:

    def __init__(self, fen):
        self.board = chess.Board()
        self.MAX_DEPTH = 60
        self.piece_values = {
            # pawn
            1:100,
            # bishop
            2:310,
            # knight
            3:300,
            # rook
            4:500,
            # queen
            5:900,
            # king
            6:99999
        }
        self.square_table = square_table = {
            1: [
                0, 0, 0, 0, 0, 0, 0, 0,
                50, 50, 50, 50, 50, 50, 50, 50,
                10, 10, 20, 30, 30, 20, 10, 10,
                5, 5, 10, 25, 25, 10, 5, 5,
                0, 0, 0, 20, 20, 0, 0, 0,
                5, -5, -10, 0, 0, -10, -5, 5,
                5, 10, 10, -20, -20, 10, 10, 5,
                0, 0, 0, 0, 0, 0, 0, 0
            ],
            2: [
                -50, -40, -30, -30, -30, -30, -40, -50,
                -40, -20, 0, 0, 0, 0, -20, -40,
                -30, 0, 10, 15, 15, 10, 0, -30,
                -30, 5, 15, 20, 20, 15, 5, -30,
                -30, 0, 15, 20, 20, 15, 0, -30,
                -30, 5, 10, 15, 15, 10, 5, -30,
                -40, -20, 0, 5, 5, 0, -20, -40,
                -50, -40, -30, -30, -30, -30, -40, -50,
            ],
            3: [
                -20, -10, -10, -10, -10, -10, -10, -20,
                -10, 0, 0, 0, 0, 0, 0, -10,
                -10, 0, 5, 10, 10, 5, 0, -10,
                -10, 5, 5, 10, 10, 5, 5, -10,
                -10, 0, 10, 10, 10, 10, 0, -10,
                -10, 10, 10, 10, 10, 10, 10, -10,
                -10, 5, 0, 0, 0, 0, 5, -10,
                -20, -10, -10, -10, -10, -10, -10, -20,
            ],
            4: [
                0, 0, 0, 0, 0, 0, 0, 0,
                5, 10, 10, 10, 10, 10, 10, 5,
                -5, 0, 0, 0, 0, 0, 0, -5,
                -5, 0, 0, 0, 0, 0, 0, -5,
                -5, 0, 0, 0, 0, 0, 0, -5,
                -5, 0, 0, 0, 0, 0, 0, -5,
                -5, 0, 0, 0, 0, 0, 0, -5,
                0, 0, 0, 5, 5, 0, 0, 0
            ],
            5: [
                -20, -10, -10, -5, -5, -10, -10, -20,
                -10, 0, 0, 0, 0, 0, 0, -10,
                -10, 0, 5, 5, 5, 5, 0, -10,
                -5, 0, 5, 5, 5, 5, 0, -5,
                0, 0, 5, 5, 5, 5, 0, -5,
                -10, 5, 5, 5, 5, 5, 0, -10,
                -10, 0, 5, 0, 0, 0, 0, -10,
                -20, -10, -10, -5, -5, -10, -10, -20
            ],
            6: [
                -30, -40, -40, -50, -50, -40, -40, -30,
                -30, -40, -40, -50, -50, -40, -40, -30,
                -30, -40, -40, -50, -50, -40, -40, -30,
                -30, -40, -40, -50, -50, -40, -40, -30,
                -20, -30, -30, -40, -40, -30, -30, -20,
                -10, -20, -20, -20, -20, -20, -20, -10,
                20, 20, 0, 0, 0, 0, 20, 20,
                20, 30, 10, 0, 0, 10, 30, 20
            ]
        }
        self.board.set_fen(fen)
        self.leaves_reached = 0


    def random_response(self):
        response = random.choice(list(self.board.legal_moves))
        return str(response)


    def material_eval(self):
        score = 0
        # iterate through the pieces
        for i in range(1, 7):
            score += len(self.board.pieces(i, chess.WHITE)) * self.piece_values[i]
            score -= len(self.board.pieces(i, chess.BLACK)) * self.piece_values[i]

        return score


    def position_eval(self):
        score = 0
       
        for i in range(1, 7):
           
            w_squares = self.board.pieces(i, chess.WHITE)
            score += len(w_squares) * self.piece_values[i]
            for square in w_squares:
                score += self.square_table[i][-square]

            b_squares = self.board.pieces(i, chess.BLACK)
            score -= len(b_squares) * self.piece_values[i]
            for square in b_squares:
                score -= self.square_table[i][square]

        return score



    def minimax(self, depth, move, maximiser):
        if depth == 0:
           
            return move, self.position_eval()

        if maximiser:
            best_move = None
            best_score = -9999

            moves = list(self.board.legal_moves)

            for move in moves:
                self.leaves_reached += 1
                self.board.push(move)
                new_move, new_score = self.minimax(depth - 1, move, False)
                if new_score > best_score:
                    best_score, best_move = new_score, move
                self.board.pop()

            return best_move, best_score

        if not maximiser:
            best_move = None
            best_score = 9999

            moves = list(self.board.legal_moves)

            for move in moves:
                self.leaves_reached += 1
                self.board.push(move)
                new_move, new_score = self.minimax(depth - 1, move, True)
                if new_score < best_score:
                    best_score, best_move = new_score, move
                self.board.pop()

            return best_move, best_score


    def alpha_beta(self, depth_neg, depth_pos, move, alpha, beta, prev_moves, maximiser):

        move_sequence = []
        if depth_neg == 0:
            move_sequence.append(move)
            return move_sequence, self.position_eval()


        moves = list(self.board.legal_moves)
      
        if not moves:
            if self.board.is_checkmate():
                if self.board.result() == "1-0":
                    move_sequence.append(move)
                    return move_sequence, 1000000
                elif self.board.result() == "0-1":
                    move_sequence.append(move)
                    return move_sequence, -1000000
            else:
                move_sequence.append(move)
                return move_sequence, 0

       
        best_move = None
        best_score = -10000001 if maximiser else 10000001
        if prev_moves and len(prev_moves) >= depth_neg:
            if prev_moves[depth_neg - 1] in moves:
                moves.insert(0, prev_moves[depth_neg - 1])


        if maximiser:
            for move in moves:
                self.leaves_reached += 1
                self.board.push(move)
                new_sequence, new_score = self.alpha_beta(depth_neg - 1, depth_pos + 1, move, alpha, beta, prev_moves, False)
                self.board.pop()
                if new_score > best_score:
                    move_sequence = new_sequence
                    best_score, best_move = new_score, move

                if new_score >= beta:
                    
                    move_sequence.append(best_move)
                    return move_sequence, best_score
                
                if new_score > alpha:
                    alpha = new_score
         
            move_sequence.append(best_move)
            return move_sequence, best_score

        if not maximiser:
            for move in moves:
                self.leaves_reached += 1

                
                self.board.push(move)
                new_sequence, new_score = self.alpha_beta(depth_neg - 1, depth_pos + 1, move, alpha, beta, prev_moves, True)
                self.board.pop()

               
                if new_score < best_score:
                    move_sequence = new_sequence
                    best_score, best_move = new_score, move

              
                if new_score <= alpha:
                    # self.check_against_best(best_move, best_score, depth_pos, False)
                    move_sequence.append(best_move)
                    return move_sequence, best_score

             
                if new_score < beta:
                    beta = new_score
            move_sequence.append(best_move)
            return move_sequence, best_score

    def calculate_minimax(self, depth):
      
        maximiser = self.board.turn

        best_move, best_score = self.minimax(depth, None, maximiser)

        return str(best_move)


    def calculate_ab(self, depth):
        maximiser = self.board.turn

        move_sequence, best_score = self.alpha_beta(depth, 0, None, -10000001, 10000001, None, maximiser)
        return str(move_sequence[-1])


    def total_leaves(self):
        leaves = self.leaves_reached
        self.leaves_reached = 0
        return leaves


    def order_moves(self):
        moves = list(self.board.legal_moves)
        scores = []
        for move in moves:
            self.board.push(move)
           
            scores.append(self.material_eval())
            self.board.pop()
        sorted_indexes = sorted(range(len(scores)), key=lambda i: scores[i], reverse=False)
        return [moves[i] for i in sorted_indexes]


    def iterative_deepening(self, depth):
       
        move_list, score  = self.alpha_beta(1, 0, None, -10000001, 10000001, None, self.board.turn)
        for i in range(2, depth + 1):
           
            move_list, score = self.alpha_beta(i, 0, None, -10000001, 10000001, move_list, self.board.turn)
     
        return str(move_list[-1])

In [ ]:
def GameDataCollector(ID,Time,gameData,temp = gameFile):
    temp = temp.append({
        'Game ID':ID,
        'Game Date':Time,
        'Game Data':gameData
    },ignore_index=True)

In [ ]:
def play_game(player1, player2, visual="svg", pause=0):
    use_svg = (visual == "svg")
    board = chess.Board()
    gameID = uuid.uuid1()
    gameTime = datetime.now().strftime("%d %m %Y-%I %M %S %p")
    gameData = dict()
    gameDataList = list()
   
    try:
        while not board.is_game_over(claim_draw=True):
            if board.turn == chess.WHITE:
                uci = player1(board)
            else:
                uci = player2(board)
            oldBoard = board.fen()
            name = who(board.turn)
            board.push_uci(uci)
            board_stop = display_board(board, use_svg)
            
            scores_w = staticAnalysis(board,'White')
            scores_b = staticAnalysis(board,'Black')
                
            moveList = board.move_stack
            LenMove= len(moveList)
            temPGame = LenMove,{'old Board': oldBoard,'move': uci,'Who': name,'new board':board.fen(),'score white':scores_w,'score black':scores_b}
            gameDataList.append(temPGame)
            GameDataCollector(gameID,gameTime,gameDataList)
            #game = chess.pgn.Game()
            html = "<h2>Chess With Deep Neural Network</h2><h3>%s is thinking...<br/><b>Move %s White : %s  Black : %s <br/> %s  Play '%s'</b><br/><h5/><br/>%s<br/>" % (who(board.turn), len(moveList),scores_w,scores_b, name, uci, board_stop)
            if visual is not None:
                if visual == "svg":
                    clear_output(wait=True)
                display(HTML(html))
                if visual == "svg":
                    time.sleep(pause)
    except KeyboardInterrupt:
        msg = "Game interrupted!"
        errors ="interrupted"
        return (None, msg, board)
    
    errors = None
    
    if board.is_checkmate():
        msg = "checkmate: " + who(not board.turn) + " wins!"
        win = who(not board.turn)

    elif board.is_stalemate():
        msg = "draw: stalemate"
        win = "stalemate"
    elif board.is_fivefold_repetition():
        msg = "draw: 5-fold repetition"
        win = "5-fold"
    elif board.is_insufficient_material():
        msg = "draw: insufficient material"
        win = "insufficient"
    elif board.can_claim_draw():
        msg = "draw: claim"
        win = "claim"
        
    if visual is not None:
        msgD = "<h2>%s<h2/>" % (msg)
        display(HTML(msgD))
        time.sleep(pause*10)
    
    gameData.update({
        'GameID':gameID,
        'GameTime':gameTime,
        'Game Results' : win,
        'Error':errors,
        'Game Messege' : msg,
        'Winning Board': board,
        'GameData': gameDataList
            })
    return gameData

In [ ]:
def best_itereation(board):
    best_int= Engine(board.fen())
    bestMove = best_int.iterative_deepening(iter_depths)
    return bestMove

In [ ]:
def random_player(board):
    move = random.choice(list(board.legal_moves))
    return move.uci()

In [ ]:
def engine_move(board):
    result = engine.play(board, chess.engine.Limit(depth=engine_depths))
    results = result.move
    return results.uci()

In [ ]:
def stockFish_engine(board):
    stockfish.set_skill_level(engine_skills)
    stf_fen = stockfish.set_fen_position(board.fen())
    stf_best_moves = stockfish.get_best_move()
    return stf_best_moves

In [ ]:
def best_score(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        # go through board and return a score
        move.score = staticAnalysis(newboard, move, board.turn)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()

In [ ]:
def nTime(n):
    for i in range(n):
        engine_depths = int(i)
        start_time = time.time()
        x = play_game(engine_move, engine_move)
        end_time = time.time()
        total = end_time - start_time
        gameFile = gameFile.append({
            'Game Id':x['GameID'],
            'Game Time':x['GameTime'],
            'Engine Depths' : i,
            'Win' : x['Game Results'],
            'Total Moves': len(x['GameData']),
            'Time Taken': total,
            'Game Data':x
        },ignore_index = True)
    fileDate = datetime.now().strftime("%d_%m_%Y-%I_%M_%S_%p")
    gameFile.to_csv(fileDate+'.csv')

In [ ]:
#play_game(best_score, engine_move)
#play_game(engine_move, engine_move)
#play_game(stockFish_engine, engine_move)
#play_game(best_itereation, engine_move)

In [ ]:
testData = pd.DataFrame()
for i in range(10):
    engine_depths = int(i)
    for j in range(20):
        engine_skills = 2**j
        start_time = time.time()
        y = play_game(stockFish_engine, engine_move)
        end_time = time.time()
        total = end_time - start_time
        testData = testData.append({
                'Game Id':y['GameID'],
                'Game Time':y['GameTime'],
                'Engine Depths' : engine_depths,
                'Engine Skills': engine_skills,
                'Win' : y['Game Results'],
                'Total Moves': len(y['GameData']),
                'Time Taken': total,
                'Game Data':y
            },ignore_index = True)

In [ ]:
testData